In [1]:
with open("input.txt") as f:
    lines = [
        line.strip() for line in f.readlines() if line.strip()
    ]

In [2]:
def rotate_clockwise(grid):
    """Rotates a 2D list clockwise"""
    return list(list(row) for row in zip(*grid[::-1]))

def rotate_string_grid_clockwise(grid):
    """Rotates a list of strings clockwise"""
    return ["".join(line) for line in rotate_clockwise(grid)]

def flip_horizontal(grid):
    """Flipes a 2D list along the vertical axis"""
    return [line[::-1] for line in grid]

def flip_vertical(grid):
    """Flipes a 2D list along the horizontal axis"""
    return list(reversed(grid))

In [3]:
from typing import List, Set

class Tile:

    def __init__(self, tile_id: int, grid: List[List[str]]):
        self.tile_id = tile_id
        self.grid = grid
        self._compute_lrtb()
        self.all_sides = {self.top, self.bottom, self.left, self.right}
        self.flip_horizontal()
        self.all_sides |= {self.top, self.bottom}
        self.flip_vertical()
        self.all_sides |= {self.left, self.right}
        self.flip_horizontal()
        self.flip_vertical()

    def _compute_lrtb(self):
        """Sets the left, right, top, bottom based off of self.grid"""
        self.top = "".join(self.grid[0])
        self.bottom = "".join(self.grid[-1])
        self.left = "".join([line[0] for line in self.grid])
        self.right = "".join([line[-1] for line in self.grid])

    def rotate_clockwise(self):
        """Rotates the tile clockwise"""
        self.grid = rotate_string_grid_clockwise(self.grid)
        self._compute_lrtb()

    def flip_horizontal(self):
        """Flips the tile along the vertical axis (i.e. left to right)"""
        self.grid = flip_horizontal(self.grid)
        self._compute_lrtb()

    def flip_vertical(self):
        """Flips along the horizontal axis"""
        self.grid = flip_vertical(self.grid)
        self._compute_lrtb()

    def any_matches(self, other_tile) -> bool:
        """Returns whether any configuration allows two tiles to line up"""
        return bool(
            self.all_sides.intersection(other_tile.all_sides)
        )

    def current_sides(self) -> Set[str]:
        """Return a set of the current sides"""
        return {self.left, self.top, self.right, self.bottom}

    def __eq__(self, other_tile):
        return self.tile_id == other_tile.tile_id

    def __repr__(self):
        return str(self.tile_id) + "\n" + "\n".join(self.grid)

In [4]:
from collections import defaultdict
import re

id_to_tile = defaultdict(list)
last_tile = None

for line in lines:

    if match := re.fullmatch("Tile (\d+):", line):
        last_tile = int(match.group(1))
    else:
        id_to_tile[last_tile].append(line)

id_to_tile = {
    tile_id: Tile(tile_id, grid) for tile_id, grid in id_to_tile.items()
}

## part1

We take a bit of a shortcut here. For each tile, we find all other tiles
that could possibly be a neighbor (must have two sides that could align).
If a tile only has 2 possible neighbors, then it must be a corner tile.

In [5]:
from functools import reduce
from operator import mul

tile_id_to_neighbors = {
    tile_id: set(
        other_tile.tile_id
        for other_tile in id_to_tile.values()
        if tile.any_matches(other_tile) and tile != other_tile
    )
    for tile_id, tile in id_to_tile.items()
}

corner_tiles = set(
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 2
)
side_tiles = set(
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 3
)
interior_tiles = set(
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 4
)

reduce(mul, (tile_id for tile_id in corner_tiles))

19955159604613

## part2

We start by picking a corner tile to begin our grid. We then have to orient
the corner tile and its two neighbors such that one borders it to the right
and one to the bottom.

From there, we iterate through the rest of the positions in the grid. If we
are on a side position, then we simply need a tile oriented to match its
neighbors.

In [6]:
DIMENSIONS = 12

grid = [[None] * DIMENSIONS for _ in range(DIMENSIONS)]

available_tile_ids = set(
    tile_id for tile_id in corner_tiles | side_tiles | interior_tiles
)

upper_left_tile = list(corner_tiles)[0]
grid[0][0] = upper_left_tile
grid[0][1], grid[1][0] = tuple(tile_id_to_neighbors[upper_left_tile])

available_tile_ids.remove(upper_left_tile)
available_tile_ids.remove(grid[0][1])
available_tile_ids.remove(grid[1][0])

Start by filling out all of the tiles in the grid, then we need to orient
them correctly.

In [7]:
CORNERS = {
    (0, 0), (0, DIMENSIONS-1), (DIMENSIONS-1, 0), (DIMENSIONS-1, DIMENSIONS-1)
}
SIDES = {0, DIMENSIONS-1}

def is_corner(row, col):
    return (row, col) in CORNERS

def is_side(row, col):
    return row in SIDES or col in SIDES and not is_corner(row, col)

def is_interior(row, col):
    return not is_corner(row, col) and not is_side(row, col)

for row in range(DIMENSIONS):
    for col in range(DIMENSIONS):

        if grid[row][col] is not None:
            continue

        if is_corner(row, col):
            if col == 0:
                neighbor = grid[row-1][col]
            else:
                neighbor = grid[row][col-1]

            candidate_tiles = corner_tiles & tile_id_to_neighbors[neighbor] & available_tile_ids

        elif is_side(row, col):
            if row == 0 or row == DIMENSIONS - 1:  # top or bottom row
                neighbor = grid[row][col-1]
            elif col == 0 or col == DIMENSIONS - 1:  # left or right side
                neighbor = grid[row-1][col]

            candidate_tiles = side_tiles & tile_id_to_neighbors[neighbor] & available_tile_ids

        else:  # interior tile
            assert is_interior
            # check tiles above and to the left, there should only be one that
            # matches both
            tile_above = grid[row-1][col]
            tile_left = grid[row][col-1]
            candidate_tiles = interior_tiles & available_tile_ids
            candidate_tiles &= tile_id_to_neighbors[tile_above]
            candidate_tiles &= tile_id_to_neighbors[tile_left]

        assert len(candidate_tiles) == 1
        this_tile = candidate_tiles.pop()
        available_tile_ids.remove(this_tile)
        grid[row][col] = this_tile

In [8]:
grid

[[2633, 2203, 2207, 1181, 1811, 2713, 2777, 2917, 3491, 2999, 2083, 1867],
 [1973, 3581, 1693, 2861, 3461, 1453, 2111, 1483, 2161, 2579, 1979, 2543],
 [3863, 2909, 3259, 1669, 2269, 3547, 2477, 3919, 1223, 1033, 2383, 1019],
 [2903, 3083, 1667, 3989, 2153, 3467, 2503, 1487, 3617, 2309, 2551, 3079],
 [1069, 2293, 2221, 2131, 1559, 3257, 3637, 3613, 3907, 2693, 3691, 3583],
 [2393, 1229, 2857, 2251, 2311, 3877, 3359, 1237, 2333, 3373, 3631, 1823],
 [3803, 3847, 3571, 3739, 1723, 1997, 3299, 3607, 1321, 1459, 2677, 2791],
 [2699, 2351, 1151, 2753, 3533, 1597, 3727, 3457, 2729, 3331, 1171, 1201],
 [1777, 2237, 3821, 3323, 1621, 1759, 3889, 1987, 3917, 3449, 1381, 1327],
 [2939, 1213, 3391, 2411, 3191, 1153, 3559, 2473, 1549, 3659, 1471, 1619],
 [3779, 2179, 3229, 1847, 1433, 2137, 3253, 2141, 2837, 2029, 1493, 3167],
 [1663, 1877, 3527, 2273, 1999, 1063, 2341, 3539, 2803, 1091, 1873, 2441]]

Now, we need to orient all tiles so that the borders line up
with neighboring tiles.

In [9]:
from copy import deepcopy

tile_grid = [[None] * DIMENSIONS for _ in range(DIMENSIONS)]

for row in range(DIMENSIONS):
    for col in range(DIMENSIONS):
        this_tile = id_to_tile[grid[row][col]]
        tile_left = tile_right = tile_above = tile_below = None
        if col > 0:
            tile_left = id_to_tile[grid[row][col-1]]
        if col < DIMENSIONS - 1:
            tile_right = id_to_tile[grid[row][col+1]]
        if row > 0:
            tile_above = id_to_tile[grid[row-1][col]]
        if row < DIMENSIONS - 1:
            tile_below = id_to_tile[grid[row+1][col]]

        orientation_found = False
        horizontal_flip = False

        while not orientation_found:

            if horizontal_flip:
                this_tile.flip_horizontal()
            else:
                this_tile.flip_vertical()

            horizontal_flip = not horizontal_flip

            for _ in range(4):
                left_found = tile_left is None or this_tile.left in tile_left.all_sides
                right_found = tile_right is None or this_tile.right in tile_right.all_sides
                top_found = tile_above is None or this_tile.top in tile_above.all_sides
                bottom_found = tile_below is None or this_tile.bottom in tile_below.all_sides

                orientation_found = left_found and right_found and top_found and bottom_found
                if orientation_found:
                    break

                this_tile.rotate_clockwise()

        tile_grid[row][col] = this_tile

In [10]:
string_grid = [[None] * DIMENSIONS for _ in range(DIMENSIONS)]

for row in range(DIMENSIONS):
    for col in range(DIMENSIONS):
        this_tile = tile_grid[row][col]
        string_grid[row][col] = [line[1:-1] for line in this_tile.grid[1:-1]]

In [11]:
final_string_grid = []

for row in string_grid:
    for i in range(len(row[0])):
        final_string_grid.append("".join([grid[i] for grid in row]))

In [12]:
first_row_regex = ".{18}#."
second_row_regex = "#.{4}##.{4}##.{4}###"
third_row_regex = ".#..#..#..#..#..#..."

candidates = [
    [char == "#" for char in line]
    for line in final_string_grid
]

for flip_func in (flip_horizontal, flip_vertical, flip_horizontal):
    for _ in range(4):
        # figure out where serpant occurs
        for row_num in range(len(final_string_grid) - 2):
            first_row = final_string_grid[row_num]
            second_row = final_string_grid[row_num+1]
            third_row = final_string_grid[row_num+2]

            for start_index in range(len(first_row)-20):
                first_str = first_row[start_index:start_index+20]
                second_str = second_row[start_index:start_index+20]
                third_str = third_row[start_index:start_index+20]

                first_row_match = re.fullmatch(first_row_regex, first_str)
                second_row_match = re.fullmatch(second_row_regex, second_str)
                third_row_match = re.fullmatch(third_row_regex, third_str)
                if first_row_match and second_row_match and third_row_match:
                    candidates[row_num][start_index + 18] = False

                    for col in (0, 5, 6, 11, 12, 17, 18, 19):
                        candidates[row_num+1][start_index + col] = False

                    for col in (1, 4, 7, 10, 13, 16):
                        candidates[row_num+2][start_index + col] = False

        final_string_grid = rotate_string_grid_clockwise(final_string_grid)
        candidates = rotate_clockwise(candidates)

    final_string_grid = flip_func(final_string_grid)
    candidates = flip_func(candidates)

In [13]:
sum(
    candidate
    for line in candidates
    for candidate in line
)

1639